In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

# Загрузка данных
df_Pl = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRKJzXJlQSNew0dxrQ8mFQMhBv_4owvfsF2If1b-rmxMZkR5gabHC4OiaSwt8Ul1Omc8taR27UohSeg/pub?gid=429486365&single=true&output=csv')

end_know_date = df_Pl['time'].iloc[-1]

end_know_date_index = df_Pl[df_Pl['time'] == end_know_date].index[0] +1

nan_indices = df_Pl.index[df_Pl['P_l'].isna()]

avg_pl = df_Pl['P_l'].mean()

end_date = df_Pl['time'].iloc[-1]

future_dates = pd.date_range(start='2023-09-10', end='2024-12-31', freq='5T')  # Измените freq по необходимости

future_df = pd.DataFrame({'time': future_dates, 'Scaled_P_l': np.nan})

df_Pl = pd.concat([df_Pl, future_df], ignore_index=True)

end_future_date = df_Pl['time'].iloc[-1]

df_Pl['P_l'].fillna(avg_pl, inplace=True)

df_Pl['time'] = pd.to_datetime(df_Pl['time'])

scaler = MinMaxScaler(feature_range=(0, 1))

df_Pl['Scaled_P_l'] = scaler.fit_transform(df_Pl['P_l'].values.reshape(-1, 1))

look_back = 300

end_date_index = df_Pl[df_Pl['time'] >= end_date].index.min()

X, y = [], []

for i in tqdm(range(end_date_index - look_back)):
    X.append(df_Pl[['Scaled_P_l', 'time']][i:(i + look_back)].values)
    y.append(df_Pl['Scaled_P_l'][i + look_back])

X, y = np.array(X), np.array(y)


100%|██████████| 762590/762590 [31:40<00:00, 401.25it/s]  


In [45]:
print(df_Pl)

                            time           P_l  Scaled_P_l
0      2016-06-10 16:50:00+00:00  26441.630000    0.247149
1      2016-06-10 16:55:00+00:00  26039.196000    0.243314
2      2016-06-10 17:00:00+00:00  21790.632000    0.202823
3      2016-06-10 17:05:00+00:00  22824.290000    0.212674
4      2016-06-10 17:10:00+00:00  23408.187000    0.218239
...                          ...           ...         ...
900551 2024-12-30 23:40:00+00:00  27261.640799    0.254965
900552 2024-12-30 23:45:00+00:00  27261.640799    0.254965
900553 2024-12-30 23:50:00+00:00  27261.640799    0.254965
900554 2024-12-30 23:55:00+00:00  27261.640799    0.254965
900555 2024-12-31 00:00:00+00:00  27261.640799    0.254965

[900556 rows x 3 columns]


In [2]:
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Input, concatenate, LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
import numpy as np

time_scaler = MinMaxScaler()

print(end_know_date_index)

train_size_percent = 0.95
train_size = int(end_know_date_index * train_size_percent)

# Проверка, чтобы train_size не превышало индекс end_know_date
train_size = min(train_size, end_know_date_index)


X_train, X_test = X[0:train_size, :], X[train_size:len(X), :]
y_train, y_test = y[0:train_size], y[train_size:len(y)]

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 2))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 2))

# Создание входов
input_cnn = Input(shape=(X_train.shape[1], 1), name='input_cnn')
input_lstm = Input(shape=(X_train.shape[1], 1), name='input_lstm')

# Сверточный слой для признака P_l
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu')(input_cnn)
pool1 = MaxPooling1D(pool_size=2)(conv1)
flat1 = Flatten()(pool1)

# Вход для времени
flat2 = Flatten()(input_lstm)

# Объединение выходов сверточного слоя и выходов для времени
merged = concatenate([flat1, flat2])

dense = Dense(units=50, activation='relu')(merged)

output = Dense(units=1, activation='linear')(dense)

model = Model(inputs=[input_cnn, input_lstm], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

X_cnn = X_train[:, :, 0:1].astype('float32')

X_lstm = np.reshape(X_train[:, :, 1], (X_train.shape[0] * X_train.shape[1], 1))
X_lstm = np.vectorize(lambda x: x.timestamp())(X_train[:, :, 1]).astype('float32')
X_lstm_scaled = time_scaler.fit_transform(X_lstm)

X_lstm = np.reshape(X_lstm_scaled, (X_train.shape[0], X_train.shape[1], 1))

end_know_date_index = df_Pl[df_Pl['time'] == end_know_date].index[0]
X_train_cnn = X_cnn[:end_know_date_index]
X_train_lstm = X_lstm[:end_know_date_index]
y_train = y[:end_know_date_index]


In [3]:
epochs = 25
batch_size = 252

model.fit([X_train_cnn, X_train_lstm], y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)


Epoch 1/25
2589/2589 [==============================] - 23s 9ms/step - loss: 0.0011 - val_loss: 3.6574e-04
Epoch 2/25
2589/2589 [==============================] - 22s 8ms/step - loss: 2.4832e-04 - val_loss: 3.9634e-04
Epoch 3/25
2589/2589 [==============================] - 22s 9ms/step - loss: 2.4136e-04 - val_loss: 4.1598e-04
Epoch 4/25
2589/2589 [==============================] - 22s 9ms/step - loss: 2.2804e-04 - val_loss: 4.6444e-04
Epoch 5/25
2589/2589 [==============================] - 23s 9ms/step - loss: 2.2135e-04 - val_loss: 4.1208e-04
Epoch 6/25
2589/2589 [==============================] - 23s 9ms/step - loss: 2.1756e-04 - val_loss: 2.7698e-04
Epoch 7/25
2589/2589 [==============================] - 23s 9ms/step - loss: 2.1332e-04 - val_loss: 2.8900e-04
Epoch 8/25
2589/2589 [==============================] - 22s 9ms/step - loss: 2.1087e-04 - val_loss: 3.5062e-04
Epoch 9/25
2589/2589 [==============================] - 22s 8ms/step - loss: 2.0919e-04 - val_loss: 2.9246e-04
Epoch

In [4]:
model.save(r'/Users/nikitasavvin/Desktop/Учеба/Load-Balance/lstm/models/' + f'model_e_{epochs}_bs_{batch_size}.h5')

/Users/nikitasavvin/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [64]:
def predict_consumption(dates_to_predict, model, X_cnn, df_Pl, scaler, look_back):
    dates_to_predict = pd.to_datetime(dates_to_predict)

    if dates_to_predict.tzinfo is None:
        dates_to_predict = dates_to_predict.tz_localize(df_Pl['time'].dt.tz)

    X_predict_cnn = []
    X_predict_lstm = []
    print(X_cnn)
    for date in dates_to_predict:
        # Получение последних look_back точек для каждой даты
        indices = df_Pl[df_Pl['time'] <= date].index[-look_back:]
        print(indices)
        X_predict_cnn.append(X_cnn[indices])
        X_predict_lstm.append(df_Pl[df_Pl['time'] <= date].tail(look_back)[['time']].values)

    X_predict_cnn = np.array(X_predict_cnn)
    X_predict_lstm = np.array(X_predict_lstm)

    X_predict_cnn = np.reshape(X_predict_cnn, (X_predict_cnn.shape[0], X_predict_cnn.shape[1], 1))

    # Преобразование времени
    X_predict_lstm[:,:,0] = (
        (X_predict_lstm[:,:,0].astype('datetime64[ns]').view('int64') -
         df_Pl['time'].min().timestamp()) /
        (df_Pl['time'].max().timestamp() - df_Pl['time'].min().timestamp())
    )

    predicted_scaled_values = model.predict([X_predict_cnn.astype('float32'), X_predict_lstm.astype('float32')])

    predicted_values = scaler.inverse_transform(predicted_scaled_values)

    df_predict = pd.DataFrame({'time': dates_to_predict, 'P_l': predicted_values.flatten()})

    return df_predict


In [62]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRKJzXJlQSNew0dxrQ8mFQMhBv_4owvfsF2If1b-rmxMZkR5gabHC4OiaSwt8Ul1Omc8taR27UohSeg/pub?gid=429486365&single=true&output=csv')


In [63]:
print(len(df))

762891


In [65]:

import matplotlib.pyplot as plt


dates_to_predict = pd.date_range(start='2023-09-08', end='2023-09-13', freq='5T')
dates_to_predict = pd.to_datetime(dates_to_predict)

# google_drive_path = '/content/gdrive/MyDrive/'
# model_path = google_drive_path + f'model_e_{epochs}_bs_{batch_size}.h5'

# model =  load_model(model_path)
model = model

predicted_df = predict_consumption(dates_to_predict, model, X_cnn, df_Pl, scaler, look_back)

print(predicted_df)

plt.figure(figsize=(14, 7))
plt.plot(predicted_df['time'], predicted_df['P_l'], linewidth=1)
plt.title(label="Predict")
plt.ylabel("P_l Value")
plt.xlabel("Time")
plt.show()

[[[0.2471494 ]
  [0.24331397]
  [0.2028228 ]
  ...
  [0.21761772]
  [0.20887344]
  [0.21284482]]

 [[0.24331397]
  [0.2028228 ]
  [0.21267413]
  ...
  [0.20887344]
  [0.21284482]
  [0.22240874]]

 [[0.2028228 ]
  [0.21267413]
  [0.218239  ]
  ...
  [0.21284482]
  [0.22240874]
  [0.23093477]]

 ...

 [[0.15074944]
  [0.15820615]
  [0.16129883]
  ...
  [0.15923735]
  [0.1602905 ]
  [0.14811806]]

 [[0.15820615]
  [0.16129883]
  [0.1529005 ]
  ...
  [0.1602905 ]
  [0.14811806]
  [0.18209448]]

 [[0.16129883]
  [0.1529005 ]
  [0.15827191]
  ...
  [0.14811806]
  [0.18209448]
  [0.2223058 ]]]
Int64Index([761547, 761548, 761549, 761550, 761551, 761552, 761553, 761554,
            761555, 761556,
            ...
            761837, 761838, 761839, 761840, 761841, 761842, 761843, 761844,
            761845, 761846],
           dtype='int64', length=300)


IndexError: index 761547 is out of bounds for axis 0 with size 724745

In [7]:
print(X_predict_cnn)

2023-09-11 15:00:00+00:00


In [7]:
X_predict_cnn